# Cohort Churn Analysis with RLM

This tutorial demonstrates how to use DSPy's **RLM (Recursive Language Model)** module to investigate a retention problem across three large DataFrames.

The RLM iteratively explores the data — computing retention curves, segmenting by acquisition channel, comparing feature usage between retained and churned users — until it identifies the root cause of churn.

## The Dataset

We use a synthetic SaaS dataset with an embedded signal:

- **10,000 users** across 5 acquisition channels
- **~250,000 feature usage events** spanning 10 feature types
- **10,000 subscription records** with churn patterns

The hidden signal the RLM should discover:
- `paid_campaign_x` has ~45% churn (vs ~15% for organic/referral)
- Churned users from that channel **never** use `advanced_reports`
- Overall churn rate is ~23.8%

## Setup

Install dependencies if needed:

In [1]:
# %pip install -q dspy pandas faker

## 1. Generate the Data

First, generate the synthetic dataset. The `generate_cohort_data.py` script creates three parquet files with carefully embedded churn signals.

Run it once — the parquet files will be reused across experiments:

In [2]:
!python generate_cohort_data.py

Generating users...
Generating subscriptions...
Generating events...

Dataset stats:
  Users:             10,000 rows
  Subscriptions:     10,000 rows
  Events:           250,918 rows
  Total memory:        16.3 MB

Embedded signal:
  paid_campaign_x churn rate: 45%
  organic churn rate:         16%

Saved to: users.parquet, subscriptions.parquet, events.parquet


## 2. Load the Data

Load the three DataFrames that the RLM will analyze:

In [3]:
import pandas as pd

users = pd.read_parquet("users.parquet")
events = pd.read_parquet("events.parquet")
subscriptions = pd.read_parquet("subscriptions.parquet")

print(f"Users:         {len(users):>10,} rows")
print(f"Events:        {len(events):>10,} rows")
print(f"Subscriptions: {len(subscriptions):>10,} rows")

users.head()

Users:             10,000 rows
Events:           250,918 rows
Subscriptions:     10,000 rows


,user_id,email,name,signup_date,acquisition_channel,plan_at_signup,country
0,1,johnsonjoshua@example.org,Brian Yang,2024-01-29,organic,free,US
1,2,garzaanthony@example.org,Jonathan Johnson,2024-01-27,paid_campaign_x,pro,US
2,3,jennifermiles@example.com,Kevin Pacheco,2024-04-18,organic,free,US
3,4,blakeerik@example.com,Christopher Bernard,2024-01-07,paid_campaign_x,pro,AU
4,5,curtis61@example.com,Lindsey Roman,2024-04-17,organic,starter,BR


In [4]:
events.head()

,user_id,event_type,timestamp,session_id,duration_seconds
0,9910,report_export,2024-01-01 02:22:19,97b17398,263
1,4854,file_upload,2024-01-01 04:24:55,cda2b255,28
2,2520,search,2024-01-01 04:34:43,f833547a,183
3,9206,report_export,2024-01-01 05:07:47,611423a4,27
4,1550,dashboard_view,2024-01-01 05:39:15,275562c2,67


In [5]:
subscriptions.head()

,user_id,subscription_start,subscription_end,plan,mrr,status,cancellation_reason
0,1,2024-01-29,NaT,free,0,active,NaN
1,2,2024-01-27,NaT,pro,79,active,NaN
2,3,2024-04-18,NaT,free,0,active,NaN
3,4,2024-01-07,NaT,pro,79,active,NaN
4,5,2024-04-17,2024-05-09,starter,29,cancelled,poor_support


## 3. Define the Signature

The signature tells the RLM what data it has and what outputs to produce. The docstring becomes the system prompt — it guides the model's investigation strategy.

Note the use of `dspy.DataFrame` as the input type. This tells the RLM to serialize the DataFrame and make it available in the code sandbox.

In [6]:
import dspy


class CohortRetentionAnalysis(dspy.Signature):
    """You are a data analyst investigating why user retention is dropping.

    You have access to three DataFrames:
    - `users`: user profiles with signup dates and acquisition channels
    - `events`: feature usage events with timestamps
    - `subscriptions`: subscription status, plan, MRR, and cancellation reasons

    Investigate the data step by step. Compute retention by cohort, segment by
    acquisition channel, compare feature usage between retained and churned users,
    and identify the root cause of churn.
    """

    users: dspy.DataFrame = dspy.InputField(
        desc="User profiles with signup_date, acquisition_channel, country"
    )
    events: dspy.DataFrame = dspy.InputField(
        desc="Feature usage events with user_id, event_type, timestamp"
    )
    subscriptions: dspy.DataFrame = dspy.InputField(
        desc="Subscription records with status, plan, mrr, cancellation_reason"
    )

    overall_churn_rate: float = dspy.OutputField(
        desc="Overall churn rate as a decimal (e.g. 0.25 for 25%)"
    )
    worst_channel: str = dspy.OutputField(
        desc="Acquisition channel with highest churn rate"
    )
    key_finding: str = dspy.OutputField(
        desc="The main insight about what differentiates churned users"
    )
    recommendations: str = dspy.OutputField(
        desc="2-3 actionable recommendations based on the analysis"
    )

## 4. Configure the LM and Run

Configure your language model and create the RLM. The `verbose=True` flag lets you watch the model's iterative investigation in real time.

The RLM will (should):
1. Examine the data schema
2. Compute overall churn metrics
3. Segment by acquisition channel
4. Compare feature usage between churned and retained users
5. Identify the root cause and formulate recommendations

In [7]:
# Configure your LM (swap the model as needed)
lm = dspy.LM("openrouter/anthropic/claude-opus-4.6", cache=False)
dspy.configure(lm=lm)

# Create the RLM analyzer
analyzer = dspy.RLM(
    CohortRetentionAnalysis,
    max_iterations=15,
    verbose=True,
)

# Run the analysis
result = analyzer(
    users=users,
    events=events,
    subscriptions=subscriptions,
)

2026/02/11 12:50:09 INFO dspy.predict.rlm: RLM iteration 1/15
Reasoning: Let me start by exploring the data to understand its structure, then compute churn rates, segment by channel, and analyze feature usage differences.
Code:
```python
# First, let's explore the data
print("=== USERS ===")
print(users.shape)
print(users.head())
print("\nAcquisition channels:", users['acquisition_channel'].value_counts())
print("\nPlan at signup:", users['plan_at_signup'].value_counts())

print("\n=== SUBSCRIPTIONS ===")
print(subscriptions.shape)
print(subscriptions.head())
print("\nStatus:", subscriptions['status'].value_counts())
print("\nPlans:", subscriptions['plan'].value_counts())
print("\nCancellation reasons:", subscriptions['cancellation_reason'].value_counts())
```
/Users/kmad/dev/dspy-fresh/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized va

## 5. Inspect the Results

The RLM returns structured outputs matching the signature's output fields:

In [17]:
import textwrap

def wrap80(text):
    return "\n  ".join(textwrap.wrap(text, width=80))

print(f"\nRLM completed in {len(result.trajectory)} iterations")
print(f"Discovered Churn Rate: {result.overall_churn_rate:.1%}")
print("Expected: 23.8%")
print(f"Worst Channel:     {result.worst_channel}")
print("=====================")
print(f"\nKey Finding:\n  {wrap80(result.key_finding)}")
print(f"\nRecommendations:\n  {wrap80(result.recommendations)}")



RLM completed in 5 iterations
Discovered Churn Rate: 23.8%
Expected: 23.8%
Worst Channel:     paid_campaign_x

Key Finding:
  The primary driver of user retention decline is the paid_campaign_x acquisition
  channel, which has a 44.8% churn rate — nearly 3x higher than all other channels
  (~15%). This single channel accounts for 46.5% of all churned users despite
  representing only 24.7% of the user base. Churned users across all channels show
  dramatically lower feature engagement (avg 12.6 events vs 29.0 for active
  users), with advanced_reports usage being the strongest predictor of retention
  (4.29x higher usage among retained users). The paid_campaign_x channel likely
  attracts low-intent users who never develop habitual product usage, leading to
  churn across all cancellation reasons roughly equally.

Recommendations:
  1. IMMEDIATE: Audit and restructure paid_campaign_x — review ad targeting,
  messaging, and landing pages to attract higher-intent users. Consider pausing

### Sample Output (Claude Opus 4.6, 6 iterations, ~90s)

```
Overall Churn Rate: 23.8%
Worst Channel:     paid_campaign_x

Key Finding:
  The primary driver of churn is low adoption of the 'advanced_reports' feature,
  especially among paid_campaign_x users. Only 48.8% of paid_campaign_x users
  adopt advanced_reports (vs ~85% for other channels), and those who don't have
  an 87.5% churn rate. Meanwhile, 100% of paid_campaign_x users who used
  advanced_reports remained active.

Recommendations:
  1. Revamp paid_campaign_x onboarding: Create a dedicated flow that guides
     users to discover and use advanced_reports within their first session.
  2. Re-evaluate paid_campaign_x targeting and messaging: The campaign attracts
     users who churn at 3x the rate of other channels.
  3. Implement an advanced_reports activation trigger for all new users who
     haven't used it within 7 days.

RLM completed in 6 iterations
```

## 6. Examine the Trajectory

The `trajectory` attribute contains each iteration's reasoning and code. This lets you see exactly how the model explored the data:

In [9]:
for i, step in enumerate(result.trajectory):
    print(f"\n{'='*60}")
    print(f"Iteration {i+1}")
    print(f"{'='*60}")
    print(f"Reasoning: {step.get('reasoning', '')[:200]}...")
    print(f"Code:\n{step.get('code', '')[:300]}...")


Iteration 1
Reasoning: Let me start by exploring the data to understand its structure, then compute churn rates, segment by channel, and analyze feature usage differences....
Code:
# First, let's explore the data
print("=== USERS ===")
print(users.shape)
print(users.head())
print("\nAcquisition channels:", users['acquisition_channel'].value_counts())
print("\nPlan at signup:", users['plan_at_signup'].value_counts())

print("\n=== SUBSCRIPTIONS ===")
print(subscriptions.shape)
...

Iteration 2
Reasoning: Now I have a good overview. Let me compute:
1. Overall churn rate
2. Churn rate by acquisition channel to find the worst channel
3. Feature usage comparison between retained and churned users
4. Cohor...
Code:
# Overall churn rate
total_users = len(subscriptions)
churned_users = len(subscriptions[subscriptions['status'] == 'cancelled'])
overall_churn_rate = churned_users / total_users
print(f"Overall churn rate: {overall_churn_rate:.4f} ({overall_churn_rate*100:.2f}%)")

# Merge users 

## Ground Truth

For reference, here's what's embedded in the synthetic data:

| Metric | Expected Value |
|--------|---------------|
| Overall churn rate | ~23.8% |
| Worst channel | `paid_campaign_x` (~45% churn) |
| Key behavioral signal | Churned `paid_campaign_x` users never use `advanced_reports` |

The churn probabilities by channel:
- `paid_campaign_x`: ~45% (3x base)
- `social`: ~22.5% (1.5x base)
- `organic`, `referral`, `paid_campaign_y`: ~15% (base)

The behavioral signal: churned users from `paid_campaign_x` are excluded from the `advanced_reports` feature in the event generation. This means the RLM needs to cross-reference feature usage against churn status against acquisition channel to find the root cause — a multi-hop analytical join.